In [1]:
!pip install pythainlp
!pip install rank_bm25

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.6/9.6 MB 6.3 MB/s eta 0:00:0000:0100:010m


In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os

from pythainlp.tokenize import word_tokenize
from pythainlp.corpus import thai_stopwords

# Prepare corpus of documents

In [3]:
doc_ls = []
context_folder = '/kaggle/input/ss3-nlp-cmsk-chatbot/context'

for dirname, _, filenames in os.walk(context_folder):
    for filename in filenames:
        filepath = os.path.join(dirname, filename)
        with open(filepath, 'r') as f:
            context = f.read()
        with open(filepath, 'r') as f:
            num_lines = len(f.readlines())
        doc_ls.append([filename, context])

doc_df = pd.DataFrame(doc_ls, columns=['file', 'context'])
doc_df['cid'] = doc_df.index

In [4]:
doc_df

,file,context,cid
0,1345137.txt,การซื้อขายตราสารหนี้ในไทย\n\n2.\tกระบวกการออกต...,0
1,1345157.txt,การวิเคราะห์และกลยุทธ์การลงทุนในกองทุนรวม\n\n3...,1
2,1345179.txt,ตราสารที่เชื่อมโยงกับตราสารทุน\n\n5.\tใบสำคัญแ...,2
3,1345178.txt,ตราสารที่เชื่อมโยงกับตราสารทุน\n\n4.\tใบสำคัญแ...,3
4,1345146.txt,การประเมินมูลค่าและอัตราผลตอบแทนของตราสารหนี้\...,4
5,1345140.txt,การซื้อขายตราสารหนี้ในไทย\n\n5.\tข้อมูลและแหล่...,5
6,1345142.txt,การประเมินมูลค่าและอัตราผลตอบแทนของตราสารหนี้\...,6
7,1345191.txt,ตลาดหลักทรัพย์แห่งประเทศไทย\n\n11.\tข้อมูลจากต...,7
8,1345193.txt,ประเภทของธุรกิจและที่มาของเงินทุนในการทำธุรกิจ...,8
9,1345149.txt,การประเมินมูลค่าหุ้นสามัญ\n\n2. วิธีการประเมิน...,9


In [5]:
th_stopwords = set(thai_stopwords())

def preprocess(text):
    tokens = word_tokenize(text, engine='newmm')
    tokens = [x for x in tokens if any(c.isalnum() for c in x)]  # eliminate spaces and symbols
    tokens = [x for x in tokens if x not in th_stopwords]
    tokens = [x for x in tokens if not x.isnumeric()]
    
    return ','.join(tokens)

In [6]:
doc_df['context_preprocess'] = doc_df['context'].apply(preprocess)

In [7]:
doc_df

,file,context,cid,context_preprocess
0,1345137.txt,การซื้อขายตราสารหนี้ในไทย\n\n2.\tกระบวกการออกต...,0,"การซื้อขาย,ตราสาร,หนี้,ไทย,กระ,บวก,ตราสาร,หนี้..."
1,1345157.txt,การวิเคราะห์และกลยุทธ์การลงทุนในกองทุนรวม\n\n3...,1,"วิเคราะห์,กลยุทธ์,การลงทุน,กองทุนรวม,แหล่งข้อม..."
2,1345179.txt,ตราสารที่เชื่อมโยงกับตราสารทุน\n\n5.\tใบสำคัญแ...,2,"ตราสาร,เชื่อมโยง,ตราสาร,ทุน,ใบสำคัญ,สิทธิ,ประโ..."
3,1345178.txt,ตราสารที่เชื่อมโยงกับตราสารทุน\n\n4.\tใบสำคัญแ...,3,"ตราสาร,เชื่อมโยง,ตราสาร,ทุน,ใบสำคัญ,สิทธิ,ซื้อ..."
4,1345146.txt,การประเมินมูลค่าและอัตราผลตอบแทนของตราสารหนี้\...,4,"การประเมิน,มูลค่า,อัตรา,ผลตอบแทน,ตราสาร,หนี้,ป..."
5,1345140.txt,การซื้อขายตราสารหนี้ในไทย\n\n5.\tข้อมูลและแหล่...,5,"การซื้อขาย,ตราสาร,หนี้,ไทย,ข้อมูล,แหล่งข้อมูล,..."
6,1345142.txt,การประเมินมูลค่าและอัตราผลตอบแทนของตราสารหนี้\...,6,"การประเมิน,มูลค่า,อัตรา,ผลตอบแทน,ตราสาร,หนี้,ก..."
7,1345191.txt,ตลาดหลักทรัพย์แห่งประเทศไทย\n\n11.\tข้อมูลจากต...,7,"ตลาดหลักทรัพย์,ประเทศ,ไทย,ข้อมูล,ตลาดหลักทรัพย..."
8,1345193.txt,ประเภทของธุรกิจและที่มาของเงินทุนในการทำธุรกิจ...,8,"ประเภท,ธุรกิจ,ที่มา,เงินทุน,ทำ,ธุรกิจ,1.1,ประเ..."
9,1345149.txt,การประเมินมูลค่าหุ้นสามัญ\n\n2. วิธีการประเมิน...,9,"การประเมิน,มูลค่า,หุ้นสามัญ,วิธี,การประเมิน,มู..."


## Initialize BM25 object with the tokenized corpus of documents

In [8]:
from rank_bm25 import BM25Okapi

corpus = list(doc_df['context_preprocess'])

tokenized_corpus = [doc.split(",") for doc in corpus]

bm25 = BM25Okapi(tokenized_corpus, k1=0.9, b=0.2)
# bm25 = BM25Okapi(tokenized_corpus)

# Retrieval with test questions with BM25

In [9]:
test_df = pd.read_csv('/kaggle/input/ss3-nlp-cmsk-chatbot/final_test.csv')
test_df

,id,question
0,1,ตลาดหลักทรัพย์ได้กำหนดราคาสูงสุดต่ำสุดของ Warr...
1,2,ลักษณะแนวทางในการระดมทุนมีอะไรบ้าง
2,3,Gross Price หรือ Dirty Price มีวิธีการจ่ายดอกเ...
3,4,รายได้จากการลงทุนต่อ (Reinvestment income) คือ...
4,5,ตัวอย่างลำดับเหตุการณ์ที่เกี่ยวข้องกับการจ่ายเ...
...,...,...
270,271,ผู้จัดอันดับกองทุนรวมที่ได้รับอนุญาตจาก ก.ล.ต....
271,272,สูตรการคำนวณ Cash Settlement คืออะไร
272,273,ข้อดีของการประเมินมูลค่าด้วย PB Ratio คืออะไร
273,274,การรวมหุ้น (Reverse Stock Split) คืออะไร


In [10]:
test_df

,id,question
0,1,ตลาดหลักทรัพย์ได้กำหนดราคาสูงสุดต่ำสุดของ Warr...
1,2,ลักษณะแนวทางในการระดมทุนมีอะไรบ้าง
2,3,Gross Price หรือ Dirty Price มีวิธีการจ่ายดอกเ...
3,4,รายได้จากการลงทุนต่อ (Reinvestment income) คือ...
4,5,ตัวอย่างลำดับเหตุการณ์ที่เกี่ยวข้องกับการจ่ายเ...
...,...,...
270,271,ผู้จัดอันดับกองทุนรวมที่ได้รับอนุญาตจาก ก.ล.ต....
271,272,สูตรการคำนวณ Cash Settlement คืออะไร
272,273,ข้อดีของการประเมินมูลค่าด้วย PB Ratio คืออะไร
273,274,การรวมหุ้น (Reverse Stock Split) คืออะไร


In [11]:
test_df['question_preprocess'] = test_df['question'].apply(preprocess)
y_pred = []

for question in test_df['question_preprocess']:
    tokenized_query = question.split(",")
    doc_scores = bm25.get_scores(tokenized_query)
    y_pred.append(np.argsort(doc_scores)[::-1][:5].tolist())

In [12]:
df_res = pd.DataFrame(y_pred, columns =['top1','top2','top3','top4','top5'])
df_res

,top1,top2,top3,top4,top5
0,26,0,33,48,10
1,18,10,8,47,30
2,34,29,0,33,13
3,22,21,19,49,36
4,17,24,39,16,7
...,...,...,...,...,...
270,25,19,15,38,1
271,24,13,48,12,44
272,46,53,15,49,24
273,10,24,39,17,57


In [13]:
top5_df = test_df
top5_df[['top1','top2','top3','top4','top5']] = df_res[['top1','top2','top3','top4','top5']]
top5_df = top5_df.drop('question_preprocess', axis=1)

In [14]:
top5_df

,id,question,top1,top2,top3,top4,top5
0,1,ตลาดหลักทรัพย์ได้กำหนดราคาสูงสุดต่ำสุดของ Warr...,26,0,33,48,10
1,2,ลักษณะแนวทางในการระดมทุนมีอะไรบ้าง,18,10,8,47,30
2,3,Gross Price หรือ Dirty Price มีวิธีการจ่ายดอกเ...,34,29,0,33,13
3,4,รายได้จากการลงทุนต่อ (Reinvestment income) คือ...,22,21,19,49,36
4,5,ตัวอย่างลำดับเหตุการณ์ที่เกี่ยวข้องกับการจ่ายเ...,17,24,39,16,7
...,...,...,...,...,...,...,...
270,271,ผู้จัดอันดับกองทุนรวมที่ได้รับอนุญาตจาก ก.ล.ต....,25,19,15,38,1
271,272,สูตรการคำนวณ Cash Settlement คืออะไร,24,13,48,12,44
272,273,ข้อดีของการประเมินมูลค่าด้วย PB Ratio คืออะไร,46,53,15,49,24
273,274,การรวมหุ้น (Reverse Stock Split) คืออะไร,10,24,39,17,57


# Save files (to be sent to QA team)

In [15]:
top5_df.to_csv('final_test_top5.csv', index=False)

In [16]:
doc_df[['file', 'cid']].to_csv('documents.csv', index=False)